In [1]:
import pandas as pd
from pathlib import Path

# Load the id_lookup_table and demographics DataFrame
id_lookup_table = pd.read_csv("/home/changbae/fmri_project/MDMR/notebook/data/id_lookup_table.csv")
demographics = pd.read_excel("/home/changbae/fmri_project/MDMR/notebook/data/demographics.xlsx", skiprows=1, header=0)

scores_xls = pd.ExcelFile("/home/changbae/fmri_project/MDMR/notebook/data/scores.xlsx")

# Load score sheets
STAI_X_1 = scores_xls.parse("STAI_X_1")
STAI_X_2 = scores_xls.parse("STAI_X_2")
HADS = scores_xls.parse("HADS")
SWLS = scores_xls.parse("SWLS")
GAD_7 = scores_xls.parse("GAD_7")
PDSS = scores_xls.parse("PDSS")
LSAS = scores_xls.parse("LSAS")
MOCI = scores_xls.parse("MOCI")
BFNE = scores_xls.parse("BFNE")
PSWQ = scores_xls.parse("PSWQ")
FCV_19S = scores_xls.parse("FCV_19S")
HANDEDNESS = scores_xls.parse("HANDEDNESS_수정")

# Rename columns
demographics.columns = [
    'GROUP', 'Exp No.', 'Chart No.', '이름', '1. SEX', '2.AGE', '3-1. EDU', '3-2. YR_EDU', 
    '4. JOB', '5. SES', '정신질환 가족/친척 유무', '참가자와 관계', '정신과입원유무', '질환명', 
    '현재 질병 유무', '병명', '과거 뇌외상/뇌질환', '정신질환 유무', '병명2', '입원횟수', 
    '첫 정신과적 입원 (몇년 전)', '첫 정신과적 입원 (입원 년도)', '발병시기 (몇년 전)', 
    '발병시기 (입원 년도)', '항정신병 약물', '향정신병약물/ 용량(mg/tab)', '용량(tab)', 
    '현 용량 시작일', '기타 약물(약품명/용량)', '총 약물 치료기간', '비고'
]

# Define the function to generate fmri codes
def generate_fmri_code(row):
    prefix = 's' if row['GROUP'] == 'EXP' else 'c'
    return f"{prefix}{int(row['Exp No.']):04d}"

# Apply the function to each row
demographics['fmri_code'] = demographics.apply(generate_fmri_code, axis=1)

# Merge the two DataFrames on fmri_code
merged_df = pd.merge(demographics, id_lookup_table, on=['fmri_code'], how='left')

# Drop rows where fmri_code is NaN
filtered_df = merged_df.dropna(subset=['fmri_code'])

# Convert 'HAID ID' to string for consistent merging
filtered_df['HAID ID'] = filtered_df['HAID ID'].astype(str)

# Function to extract data from each score sheet
def extract_scores(sheet, score_columns):
    scores_dict = {}
    sheet_filtered = sheet[sheet['round'] == 1]
    for _, row in sheet_filtered.iterrows():
        haid_id = str(row['id'])
        if haid_id not in scores_dict:
            scores_dict[haid_id] = {}
        for col in score_columns:
            scores_dict[haid_id][col] = row[col]
    return scores_dict

# Extract data from each sheet
score_data = {
    'STAI-X-1': extract_scores(STAI_X_1, ['STAI-X-1']),
    'STAI-X-2': extract_scores(STAI_X_2, ['STAI-X-2']),
    'HADS_anxiety': extract_scores(HADS, ['HADS_anxiety']),
    'HADS_depression': extract_scores(HADS, ['HADS_depression']),
    'SWLS': extract_scores(SWLS, ['SWLS']),
    'GAD-7': extract_scores(GAD_7, ['GAD-7']),
    'PDSS': extract_scores(PDSS, ['PDSS']),
    'performance_lsas': extract_scores(LSAS, ['performance_lsas']),
    'social_interaction_lsas': extract_scores(LSAS, ['social_interaction_lsas']),
    'lsas': extract_scores(LSAS, ['lsas']),
    'MOCI': extract_scores(MOCI, ['MOCI']),
    'checking': extract_scores(MOCI, ['checking']),
    'cleaning': extract_scores(MOCI, ['cleaning']),
    'doubting': extract_scores(MOCI, ['doubting']),
    'slowness': extract_scores(MOCI, ['slowness']),
    'BFNE': extract_scores(BFNE, ['BFNE']),
    'PSWQ': extract_scores(PSWQ, ['PSWQ']),
    'Handedness(true)': extract_scores(HANDEDNESS, ['Handedness(true)']),
    'FCV-19S': extract_scores(FCV_19S, ['FCV-19S'])
}

# Add scores to the filtered dataframe
for haid_id in filtered_df['HAID ID']:
    for key, data in score_data.items():
        if haid_id in data:
            filtered_df.loc[filtered_df['HAID ID'] == haid_id, key] = data[haid_id][key]

# Select the final columns to keep in the resulting DataFrame
final_columns = [
    '1. SEX', '2.AGE', '3-2. YR_EDU', 'fmri_code', 'Screening #', 'Enrollment #', 'HAID ID',
    'STAI-X-1', 'STAI-X-2', 'HADS_anxiety', 'HADS_depression', 'SWLS', 'GAD-7', 'PDSS', 'performance_lsas', 
    'social_interaction_lsas', 'lsas', 'MOCI', 'checking', 'cleaning', 'doubting', 'slowness', 'BFNE', 
    'PSWQ', 'Handedness(true)', 'FCV-19S'
]

final_df = filtered_df[final_columns]


FileNotFoundError: [Errno 2] No such file or directory: '/home/changbae/fmri_project/MDMR/notebook/data/id_lookup_table.csv'

In [65]:
final_df.to_csv("/home/changbae/fmri_project/MDMR/notebook/data/participant_demo_clinical.csv")

In [66]:
# 각 열의 결측치 개수를 계산
missing_values_count = final_df.isnull().sum()

# 결측치 개수 출력
print("Missing values count for each column:")
print(missing_values_count)

Missing values count for each column:
1. SEX                      0
2.AGE                       0
3-2. YR_EDU                 3
fmri_code                   0
Screening #                46
Enrollment #               46
HAID ID                     0
STAI-X-1                   53
STAI-X-2                   53
HADS_anxiety               54
HADS_depression            54
SWLS                       54
GAD-7                      54
PDSS                       54
performance_lsas           54
social_interaction_lsas    54
lsas                       54
MOCI                       54
checking                   54
cleaning                   54
doubting                   54
slowness                   54
BFNE                       54
PSWQ                       54
Handedness(true)           53
FCV-19S                    54
dtype: int64


In [68]:
import pandas as pd

# Load the id_lookup_table and demographics DataFrame
id_lookup_table = pd.read_csv("/home/changbae/fmri_project/MDMR/notebook/data/id_lookup_table.csv")

# Load the scores data from the Excel file
scores_xls = pd.ExcelFile("/home/changbae/fmri_project/MDMR/notebook/data/scores.xlsx")

# STAI-X-1
STAI_X_1 = scores_xls.parse("STAI_X_1")

# STAI-X-2
STAI_X_2 = scores_xls.parse("STAI_X_2")

# HADS_anxiety, HADS_depression
HADS = scores_xls.parse("HADS")

# SWLS
SWLS = scores_xls.parse("SWLS")

# GAD_7
GAD_7 = scores_xls.parse("GAD_7")

# PDSS
PDSS = scores_xls.parse("PDSS")

# performance_lsas, social_interaction_lsas, lsas
LSAS = scores_xls.parse("LSAS")

# MOCI, checking, cleaning, doubting, slowness
MOCI = scores_xls.parse("MOCI")

# BFNE
BFNE = scores_xls.parse("BFNE")

# PSWQ
PSWQ = scores_xls.parse("PSWQ")

# FCV_19S
FCV_19S = scores_xls.parse("FCV_19S")

# Handedness(true)
HANDEDNESS = scores_xls.parse("HANDEDNESS_수정")

# Get unique ids from id_lookup_table
lookup_unique_ids = set(id_lookup_table['HAID ID'].astype(str).unique())
print(f"Unique ID count in id_lookup_table: {len(lookup_unique_ids)}")

# Function to get unique ids from a sheet
def get_unique_ids(sheet):
    return set(sheet['id'].astype(str).unique())

# List of score sheets
score_sheets = [
    ('STAI_X_1', STAI_X_1),
    ('STAI_X_2', STAI_X_2),
    ('HADS_anxiety', HADS),
    ('HADS_depression', HADS),
    ('SWLS', SWLS),
    ('GAD_7', GAD_7),
    ('PDSS', PDSS),
    ('performance_lsas', LSAS),
    ('social_interaction_lsas', LSAS),
    ('lsas', LSAS),
    ('MOCI', MOCI),
    ('checking', MOCI),
    ('cleaning', MOCI),
    ('doubting', MOCI),
    ('slowness', MOCI),
    ('BFNE', BFNE),
    ('PSWQ', PSWQ),
    ('Handedness(true)', HANDEDNESS),
    ('FCV_19S', FCV_19S)
]

# Check each sheet for unique ids and matches with id_lookup_table
for sheet_name, sheet in score_sheets:
    sheet_unique_ids = get_unique_ids(sheet)
    common_ids = sheet_unique_ids.intersection(lookup_unique_ids)
    print(f"\nSheet: {sheet_name}")
    print(f"Unique ID count in sheet: {len(sheet_unique_ids)}")
    print(f"Common ID count with id_lookup_table: {len(common_ids)}")


Unique ID count in id_lookup_table: 119

Sheet: STAI_X_1
Unique ID count in sheet: 160
Common ID count with id_lookup_table: 112

Sheet: STAI_X_2
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: HADS_anxiety
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: HADS_depression
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: SWLS
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: GAD_7
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: PDSS
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: performance_lsas
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: social_interaction_lsas
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: lsas
Unique ID count in sheet: 159
Common ID count with id_lookup_table: 112

Sheet: MOCI
Unique ID count in sheet: 159
Common I

In [69]:
# Convert 'HAID ID' to string for consistent merging
filtered_df['HAID ID'] = filtered_df['HAID ID'].astype(str)

In [73]:
scores_xls = pd.ExcelFile("/home/changbae/fmri_project/MDMR/notebook/data/scores.xlsx")

In [113]:
# Assuming filtered_df is already defined and the dataframes for each sheet are already loaded

# Define a function to extract scores
def extract_scores(df, columns, id):
    extracted_col = df[(df['id'] == id) & (df['round'] == 1)][columns]
    if len(extracted_col.values) == 0:
        return "n/a"
    else:
        return extracted_col.values[0][0]

# Dictionary to hold the extracted scores for each id
extracted_scores = []

# Loop through each unique HAID ID and extract the scores for each sheet
for id in filtered_df['HAID ID'].unique():
    target_filtered_df = filtered_df[filtered_df['HAID ID'] == id]
    score_dict = {
        'HAID ID': id,
        'GROUP': target_filtered_df['GROUP'].values[0],
        'Exp No.': target_filtered_df['Exp No.'].values[0],
        '1. SEX': target_filtered_df['1. SEX'].values[0],
        '2.AGE': target_filtered_df['2.AGE'].values[0],
        '3-2. YR_EDU': target_filtered_df['3-2. YR_EDU'].values[0],
        'fmri_code': target_filtered_df['fmri_code'].values[0],
        'Screening #': target_filtered_df['Screening #'].values[0],
        'Enrollment #': target_filtered_df['Enrollment #'].values[0],

        
        'STAI-X-1': extract_scores(STAI_X_1, ['STAI-X-1'], id),
        'STAI-X-2': extract_scores(STAI_X_2, ['STAI-X-2(true)'], id),
        'HADS_anxiety': extract_scores(HADS, ['HADS_anxiety'], id),
        'HADS_depression': extract_scores(HADS, ['HADS_depression'], id),
        'SWLS': extract_scores(SWLS, ['SWLS(true)'], id),
        'GAD-7': extract_scores(GAD_7, ['GAD-7(true변환)'], id),
        'PDSS': extract_scores(PDSS, ['PDSS'], id),
        'LSAS_performance': extract_scores(LSAS, ['performance_lsas'], id),
        'LSAS_social_interaction': extract_scores(LSAS, ['social_interaction_lsas'], id),
        'LSAS': extract_scores(LSAS, ['lsas'], id),
        'MOCI': extract_scores(MOCI, ['MOCI'], id),
        'MOCI_checking': extract_scores(MOCI, ['checking'], id),
        'MOCI_cleaning': extract_scores(MOCI, ['cleaning'], id),
        'MOCI_doubting': extract_scores(MOCI, ['doubting'], id),
        'MOCI_slowness': extract_scores(MOCI, ['slowness'], id),
        'BFNE': extract_scores(BFNE, ['BFNE(ture)'], id),
        'PSWQ': extract_scores(PSWQ, ['PSWQ(true)'], id),
        'Handedness(true)': extract_scores(HANDEDNESS, ['Handedness(true)'], id),
        'FCV-19S': extract_scores(FCV_19S, ['FCV(ttrue)'], id)
    }
    extracted_scores.append(score_dict)

In [114]:
filtered_df.columns

Index(['GROUP', 'Exp No.', 'Chart No.', '이름_x', '1. SEX', '2.AGE', '3-1. EDU',
       '3-2. YR_EDU', '4. JOB', '5. SES', '정신질환 가족/친척 유무', '참가자와 관계',
       '정신과입원유무', '질환명', '현재 질병 유무', '병명', '과거 뇌외상/뇌질환', '정신질환 유무', '병명2',
       '입원횟수', '첫 정신과적 입원 (몇년 전)', '첫 정신과적 입원 (입원 년도)', '발병시기 (몇년 전)',
       '발병시기 (입원 년도)', '항정신병 약물', '향정신병약물/ 용량(mg/tab)', '용량(tab)', '현 용량 시작일',
       '기타 약물(약품명/용량)', '총 약물 치료기간', '비고', 'fmri_code', 'Unnamed: 0',
       'Screening #', 'Enrollment #', 'HAID ID', '이름_y', 'STAI-X-1',
       'STAI-X-2', 'HADS_anxiety', 'HADS_depression', 'SWLS', 'GAD-7', 'PDSS',
       'performance_lsas', 'social_interaction_lsas', 'lsas', 'MOCI',
       'checking', 'cleaning', 'doubting', 'slowness', 'BFNE', 'PSWQ',
       'Handedness(true)', 'FCV-19S'],
      dtype='object')

In [115]:
extracted_scores_df = pd.DataFrame(extracted_scores)

In [116]:
extracted_scores_df

,HAID ID,GROUP,Exp No.,1. SEX,2.AGE,3-2. YR_EDU,fmri_code,Screening #,Enrollment #,STAI-X-1,...,LSAS,MOCI,MOCI_checking,MOCI_cleaning,MOCI_doubting,MOCI_slowness,BFNE,PSWQ,Handedness(true),FCV-19S
0,0827jck,EXP,7,2,20,12.0,s0007,1.0,EXP-0001,56.0,...,98,19,4,8,5,4,56,58,48,7
1,na0840,EXP,5,2,20,12.0,s0005,2.0,EXP-0002,67.0,...,80,15,5,6,3,2,57,62,40,18
2,spwls915,EXP,4,2,21,15.0,s0004,3.0,EXP-0003,65.0,...,95,18,6,10,1,5,59,60,40,5
3,wkddbswns,HC,9,1,25,15.0,c0009,4.0,HC-0001,35.0,...,18,24,7,11,3,7,35,47,47,2
4,bje5409,EXP,12,2,30,12.0,s0012,5.0,EXP-0004,55.0,...,57,26,8,11,4,7,51,50,47,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,hong94,HC,552,1,29,15.0,c0552,151.0,HC-0071,30.0,...,13,2,2,0,0,0,29,33,43,0
115,sunset,HC,546,1,29,16.0,c0546,152.0,HC-0072,43.0,...,19,8,2,1,4,1,40,30,40,2
116,bpark0718,HC,547,2,28,12.0,c0547,154.0,HC-0073,32.0,...,45,8,3,0,3,2,43,25,42,2
117,robin4760,HC,518,2,25,12.0,c0518,156.0,HC-0074,32.0,...,8,5,1,0,4,0,38,37,41,4


In [117]:
extracted_scores_df.to_csv("/home/changbae/fmri_project/MDMR/notebook/data/participant_demo_clinical.csv")